# **순환 신경망**

In [2]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.datasets import imdb

In [3]:
(x_train_all, y_train_all) , (x_test, y_test) = imdb.load_data(skip_top=20, num_words=100)
#매개변수 skip_top: 가장 많이 등장한 단어(ex: a, is, the...등 분석에 유용하지 않은 것들) 중 건너뛸 단어의 개수
#매개변수 num_words: 훈련에 사용할 단어의 개수

17464789/17464789 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [5]:
print(x_train_all.shape, y_train_all.shape)

(25000,) (25000,)


In [6]:
print(x_train_all[0])

[2, 2, 22, 2, 43, 2, 2, 2, 2, 65, 2, 2, 66, 2, 2, 2, 36, 2, 2, 25, 2, 43, 2, 2, 50, 2, 2, 2, 35, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 39, 2, 2, 2, 2, 2, 2, 38, 2, 2, 2, 2, 50, 2, 2, 2, 2, 2, 2, 22, 2, 2, 2, 2, 2, 22, 71, 87, 2, 2, 43, 2, 38, 76, 2, 2, 2, 2, 22, 2, 2, 2, 2, 2, 2, 2, 2, 2, 62, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 66, 2, 33, 2, 2, 2, 2, 38, 2, 2, 25, 2, 51, 36, 2, 48, 25, 2, 33, 2, 22, 2, 2, 28, 77, 52, 2, 2, 2, 2, 82, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 36, 71, 43, 2, 2, 26, 2, 2, 46, 2, 2, 2, 2, 2, 2, 88, 2, 2, 2, 2, 98, 32, 2, 56, 26, 2, 2, 2, 2, 2, 2, 2, 22, 21, 2, 2, 26, 2, 2, 2, 30, 2, 2, 51, 36, 28, 2, 92, 25, 2, 2, 2, 65, 2, 38, 2, 88, 2, 2, 2, 2, 2, 2, 2, 2, 32, 2, 2, 2, 2, 2, 32]


In [7]:
#0,1,2 제외
for i in range(len(x_train_all)) :
  x_train_all[i] = [ w for w in x_train_all[i] if w>2 ]

print(x_train_all[0])

[22, 43, 65, 66, 36, 25, 43, 50, 35, 39, 38, 50, 22, 22, 71, 87, 43, 38, 76, 22, 62, 66, 33, 38, 25, 51, 36, 48, 25, 33, 22, 28, 77, 52, 82, 36, 71, 43, 26, 46, 88, 98, 32, 56, 26, 22, 21, 26, 30, 51, 36, 28, 92, 25, 65, 38, 88, 32, 32]


In [17]:
#어휘사전 내려받기(정수->영단어) / get_word_index(): { '영단어':정수 } 딕셔너리 반환
word_to_index = imdb.get_word_index()
print(word_to_index['movie'])

index_to_word = {word_to_index[k]: k for k in word_to_index}
print(index_to_word[17])

for w in x_train_all[0] :
  print(index_to_word[w-3], end=' ')

17
movie
film just story really they you just there an from so there film film were great just so much film would really at so you what they if you at film have been good also they were just are out because them all up are film but are be what they have don't you story so because all all 

In [23]:
#1:긍정, 0:부정
print(y_train_all[:10])

[1 0 0 1 0 0 1 0 1 0]


In [25]:
np.random.seed(42)
random_index = np.random.permutation(25000) #25000 이하의 순서 랜덤

x_train = x_train_all[random_index[:20000]]
y_train = y_train_all[random_index[:20000]]
x_val = x_train_all[random_index[20000:]]
y_val = y_train_all[random_index[20000:]]

In [20]:
#각 리뷰의 길이 - 차이가 있음
print(len(x_train_all[0]), len(x_train_all[1]))

59 32


In [28]:
#길이 맞추기
from tensorflow.keras.preprocessing import sequence

maxlen = 100
x_train_seq = sequence.pad_sequences(x_train, maxlen=maxlen)
x_val_seq = sequence.pad_sequences(x_val, maxlen=maxlen)

print(x_train_seq.shape, x_val_seq.shape)
print(x_train_seq[0])

(20000, 100) (5000, 100)
[ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0 35 40 27 28 40 22 83 31 85 45
 24 23 31 70 31 76 30 98 32 22 28 51 75 56 30 33 97 53 38 46 53 74 31 35
 23 34 22 58]


In [30]:
#원-핫 인코딩하기
from tensorflow.keras.utils import to_categorical
x_train_onehot = to_categorical(x_train_seq)
x_val_onehot = to_categorical(x_val_seq)

print(x_train_onehot.shape)

(20000, 100, 100)


In [ ]:
#순환 신경망 클래스 구현하기
class RecurrentNetwork :

  def __init__(self, n_cells=10, batch_size=32, learning_rate=0,1) :
    self.n_cells = n_cells #셀 개수(순환층 뉴런 개수)
    self.batch_size = batch_size
    self.w1h = None #은닉상태에 대한 가중치
    self.w1x = None #입력에 대한 가중치
    self.b1 = None #순환층의 절편
    self.w2 = None #출력층의 가중치
    self.b2 = None #출력층의 절편
    self.h = None #순환층의 활성화 출력 저장
    self.losses = []
    self.val_losses = []
    self.lr = learning_rate


  def forpass(self, x) :
    self.h = [ np.zeros((x.shape[0], self.n_cells)) ] #은닉상태 초기화
    seq = np.swapaxes(x, 0, 1) #샘플개수 - 타임스텝 차원 바꿈
    for x in seq : #모든 단어(원핫코딩)에 대하여
      z1 = np.dot(x, self.w1x) + np.dot(self.h[-1], self.w1h) + self.b1 #순환층 선형 계산(X*Wx + H*Wh + b1)
      h = np.tanh(z1) #순환층 활성화 출력
      self.h.append(h) #활성화 출력값, 즉 전 타임스텝 은닉상태값 저장
      z2 = np.dot(h, self.w2) + self.b2 #출력층 선형 계산
    return z2


  def backprop(self, x, err) :
    m = len(x)

    w2_grad = np.dot(self.h[-1].T, err) / m #출력층 가중치 그레이디언트
    b2_grad = np.sum(err) / m #출력층 절편 그레이디언트

    seq = np.swapaxes(x, 0, 1) #샘플개수-타임스텝 차원 바꿈

    w1h_grad = w1x_grad = b1_grad = 0 #순환층 가중치들 초기화
    err_to_cell = np.dot(err, self.w2.T) * (1 - self.h[-1] ** 2) #순환층 전까지 미분값(그레이디언트값)
    for x, h in zip( seq[::-1][:10], self.h[:-1][::-1][:10] ) : #최근 타임스텝부터(뒤집기), 10개 타임스텝만 / 직전 은닉상태는 err_to_cell에 썼으므로 1개뺴고
      w1h_grad += np.dot(h.T, err_to_cell) #은닉상태 가중치 추가
      w1x_grad += np.dot(x.T, err_to_cell) #입력 가중치 추가
      b1_grad += np.sum(err_to_cell, axis=0) #순환층 절편 추가
      err_to_cell = np.dot(err_to_cell, self.w1h) * (1 - h ** 2) #이전 타임스텝의 순환층_전_그레이디언트값 계산

    w1h_grad /= m
    w1x_grad /= m
    b1_grad /= m

    return w1h_grad, w1x_grad, b1_grad, w2_grad, b2_grad


  def sigmoid(self, z) :
    a = 1 / (1 + np.exp(-z))
    return a


  def init_weights(self, n_features, n_classes) :
    orth_init = tf.initializers.Orthogonal() #순환 신경망에서 쓸 직교 행렬 초기화
    glorot_init = tf.initializers.GlorotUniform() #출력층에서 쓸 글로럿 초기화

    self.w1h = orth_init( (self.n_cells, self.n_cells) ).numpy() #은닉상태 가중치 - (셀 개수, 셀 개수)
    self.w1x = glorot_init( (n_features, self.n_cells) ).numpy() #입력 가중치 - (특성개수, 셀 개수)
    self.b1 = np.zeros( self.n_cells ) #순환층 절편 - 셀 개수
    self.w2 = glorot_init( (self.n_cells, n_classes) ).numpy() #출력층 가중치 - (셀 개수, 클래스 개수)
    self.b2 = np.zeros( n_classes ) #출력층 절편 - 클래스 개수


  def fit(self, x, y, epochs=100, x_val=None, y_val=None) :
    y = y.reshape(-1,1)
    y_val = y_val.reshape(-1,1)
    np.random.seed(42)
    self.init_weights(x.shape[2], y.shape[1]) #가중치 초기화
    for i in range(epochs) :
      print('에포크', i, end=' ')
      batch_losses = []
      for x_batch, y_batch in self.gen_batch(x,y) :
        print('.', end='')
        a = self.training(x_batch, y_batch)
        a = np.clip(a, 1e-10, 1-1e-10)
